# TensorFlow Assignment: Multilayer Perceptron (MLP) and Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Jiangyan Shao

Now that you've run through a simple logistic regression model on MNIST, let's see if we can do better (Hint: we can). For this assignment, you'll build a multilayer perceptron (MLP) and a convolutional neural network (CNN), two popular types of neural networks, and compare their performance. Some potentially useful code:

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Helper functions for creating weight variables
def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Tensorflow Functions that might also be of interest:
# tf.nn.sigmoid()
# tf.nn.relu()

### Multilayer Perceptron

Build a multilayer perceptron for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> fully connected (500 hidden units) -> nonlinearity (Sigmoid/ReLU) -> fully connected (10 hidden units) -> softmax

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

### MPL with Relu

In [26]:
import numpy as np
# Model Inputs
n_inputs = 28*28
n_classes = 10
x = tf.placeholder(tf.float32, [None, n_inputs], name='x')
y_ = tf.placeholder(tf.float32, [None, n_classes], name='y')

# Define the graph
n_hidden1 = 500
n_hidden2 = 200

def fc(x, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(x.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        w = tf.Variable(init, name='weights')
        b = tf.Variable(tf.zeros([n_neurons]), name='biases')
        z = tf.matmul(x,w) + b
        if activation == 'relu':
            return tf.nn.relu(z)
        else:
            return z

with tf.name_scope('mpl'):
    fc1 = fc(x, n_hidden1, 'fc1', activation='relu')
    fc2 = fc(fc1, n_hidden2, 'fc2', activation='relu')
    y_mlp = fc(fc2, n_classes, 'y_mlp')
    
### Create your MLP here##
### Make sure to name your MLP output as y_mlp ###
# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_mlp, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(4000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(100)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.117
step 250, validation accuracy 0.936
step 500, validation accuracy 0.958
step 750, validation accuracy 0.952
step 1000, validation accuracy 0.958
step 1250, validation accuracy 0.963
step 1500, validation accuracy 0.976
step 1750, validation accuracy 0.978
step 2000, validation accuracy 0.975
step 2250, validation accuracy 0.976
step 2500, validation accuracy 0.975
step 2750, validation accuracy 0.963
step 3000, validation accuracy 0.98
step 3250, validation accuracy 0.973
step 3500, validation accuracy 0.987
step 3750, validation accuracy 0.975
test accuracy 0.9625


### MLP with Sigmoid

In [27]:
def fc(x, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(x.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        w = tf.Variable(init, name='weights')
        b = tf.Variable(tf.zeros([n_neurons]), name='biases')
        z = tf.matmul(x,w) + b
        if activation == 'sigmoid':
            return tf.nn.sigmoid(z)
        else:
            return z

with tf.name_scope('mpl'):
    fc1 = fc(x, n_hidden1, 'fc1', activation='sigmoid')
    fc2 = fc(fc1, n_hidden2, 'fc2', activation='sigmoid')
    y_mlp = fc(fc2, n_classes, 'y_mlp')
    
### Create your MLP here##
### Make sure to name your MLP output as y_mlp ###
# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_mlp, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(4000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(100)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.11
step 250, validation accuracy 0.874
step 500, validation accuracy 0.908
step 750, validation accuracy 0.904
step 1000, validation accuracy 0.934
step 1250, validation accuracy 0.922
step 1500, validation accuracy 0.93
step 1750, validation accuracy 0.935
step 2000, validation accuracy 0.943
step 2250, validation accuracy 0.946
step 2500, validation accuracy 0.947
step 2750, validation accuracy 0.955
step 3000, validation accuracy 0.948
step 3250, validation accuracy 0.961
step 3500, validation accuracy 0.944
step 3750, validation accuracy 0.954
test accuracy 0.9571


#### Comparison

How do the sigmoid and rectified linear unit (ReLU) compare?

***

Relu provides ~0.5% more (0.9625) accurate results within 4000 iterations for this case, compared with Sigmoid activation function (0.9571) for hidden layers. The model with Relu also shows faster learning performance compare with the model with Sigmoid function (e.g step 500, Relu-0.958|Sigmoid-0.908).

***

### Convolutional Neural Network

Build a simple 2-layer CNN for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> CNN (32 5x5 filters) -> nonlinearity (ReLU) ->  (2x2 max pool) -> CNN (64 5x5 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> fully connected (1024 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax

Some additional functions that you might find helpful:

In [31]:
# Convolutional neural network functions
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Tensorflow Function that might also be of interest:
# tf.reshape()

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

*Hint: Convolutional Neural Networks are spatial models. You'll want to transform the flattened MNIST vectors into images for the CNN. Similarly, you might want to flatten it again sometime before you do a softmax. You also might want to look into the  [conv2d() documentation](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d) to see what shape kernel/filter it's expecting.*

In [34]:
# Model Inputs
n_inputs = 28*28
n_classes = 10
x = tf.placeholder(tf.float32, [None, n_inputs], name='x')
y_ = tf.placeholder(tf.float32, [None, n_classes], name='y')

# Define the graph
### Create your CNN here##
### Make sure to name your CNN output as y_conv ###
def conv2d_relu(x, W, b):
    x = tf.nn.conv2d(x,W, strides=[1,1,1,1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)
    
def flatten(x):
    flat_shape = np.array(x.get_shape().as_list()[1:]).prod()
    return tf.reshape(x, [-1, flat_shape])

def fully_conn(x, size, b):
    in_shape = int(x.get_shape()[1])
    W = tf.Variable(tf.random_normal([in_shape, size], stddev=0.01))
    b = tf.Variable(tf.constant(0.1, shape=[size]))
    return tf.matmul(x,W) + b 

def conv_net(x, W, b):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # conv-1
    conv1 = conv2d_relu(x, W['wc1'], b['bc1'])
    conv1 = max_pool_2x2(conv1)
    
    # conv-2
    conv2 = conv2d_relu(conv1, W['wc2'], b['bc2'])
    conv2 = max_pool_2x2(conv2)
    
     # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

y_conv = conv_net(x, weights, biases)

# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(10000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(50)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.102
step 250, validation accuracy 0.654
step 500, validation accuracy 0.78
step 750, validation accuracy 0.81
step 1000, validation accuracy 0.862
step 1250, validation accuracy 0.87
step 1500, validation accuracy 0.86
step 1750, validation accuracy 0.924
step 2000, validation accuracy 0.896
step 2250, validation accuracy 0.92
step 2500, validation accuracy 0.934
step 2750, validation accuracy 0.912
step 3000, validation accuracy 0.892
step 3250, validation accuracy 0.926
step 3500, validation accuracy 0.936
step 3750, validation accuracy 0.928
step 4000, validation accuracy 0.922
step 4250, validation accuracy 0.934
step 4500, validation accuracy 0.956
step 4750, validation accuracy 0.932
step 5000, validation accuracy 0.948
step 5250, validation accuracy 0.938
step 5500, validation accuracy 0.958
step 5750, validation accuracy 0.958
step 6000, validation accuracy 0.956
step 6250, validation accuracy 0.952
step 6500, validation accuracy 0.938
step 6750, v

Some differences from the logistic regression model to note:

- The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

- The logistic regression model we used previously was pretty basic, and as such, we were able to get away with using the GradientDescentOptimizer, which performs implements the gradient descent algorithm. For more difficult optimization spaces (such as the ones deep networks pose), we might want to use more sophisticated algorithms. Prof David Carlson has a lecture on this later.
    
- Because of the larger size of our network, notice that our minibatch size has shrunk.
    
- We've added a validation step every 250 minibatches. This let's us see how our model is doing during the training process, rather than sit around twiddling our thumbs and hoping for the best when training finishes. This becomes especially significant as training regimens start approaching days and weeks in length. Normally, we validate on the entire validation set, but for the sake of time we'll just stick to 10 validation minibatches (500 images) for this homework assignment.

#### Comparison

How do the MLP and CNN compare in accuracy? Training time? Why would you use one vs the other? Is there a problem you see with MLPs when applied to other image datasets?

***

[Your answer here]

***